# Data Exploration for GuitarTab Project

This notebook explores the Guitar Chords V2 and GuitarSet datasets to understand their structure, distribution, and characteristics.

## Table of Contents
1. [Import Libraries](#Import-Libraries)
2. [Define Paths and Parameters](#Define-Paths-and-Parameters)
3. [Load Sample Audio Files](#Load-Sample-Audio-Files)
4. [Visualize Waveforms](#Visualize-Waveforms)
5. [Compute and Visualize Spectrograms](#Compute-and-Visualize-Spectrograms)
6. [Class Distribution](#Class-Distribution)
7. [Conclusion](#Conclusion)

In [ ]:
# 1. Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
from src.data_preprocessing import load_wav_16k_mono, preprocess_audio, compute_spectrogram, plot_waveform, plot_spectrogram
import seaborn as sns
import pandas as pd

## 2. Define Paths and Parameters

Setting the paths to the datasets and defining any necessary parameters.

In [ ]:
# 2. Define Paths and Parameters
GUITARCHORDS_PATH = '/content/drive/MyDrive/GuitarTab/data/raw/GuitarChordsV2/'
GUITARSET_PATH = '/content/drive/MyDrive/GuitarTab/data/raw/GuitarSet/'

chords = ['Am', 'Bb', 'Bdim', 'C', 'Dm', 'Em', 'F', 'G']

## 3. Load Sample Audio Files

Load a few sample audio files from each dataset to inspect their content.

In [ ]:
# 3. Load Sample Audio Files
sample_chord = 'Am'
sample_file = os.path.join(GUITARCHORDS_PATH, sample_chord, 'Am_AcousticPlug26_1.wav')  # need to double check again

wav = load_wav_16k_mono(sample_file)
print(f"Audio Loaded: {sample_file}, Duration: {len(wav)/16000:.2f} seconds")

In [ ]:
def standardize_audio_length(wav, sr=16000, duration=3.0):
    """
    Standardize audio to a fixed length (3 seconds by default)
    """
    target_length = int(duration * sr)
    if len(wav) > target_length:
        wav = wav[:target_length]
    else:
        # Pad with zeros if too short
        wav = np.pad(wav, (0, max(0, target_length - len(wav))))
    return wav

## 4. Visualize Waveforms

Plot the waveform of the sample audio to understand its amplitude variations over time.

In [ ]:
# 4. Visualize Waveforms
plot_waveform(wav, title=f"Waveform of {sample_chord} Chord")

## 5. Compute and Visualize Spectrograms

Convert the audio waveform into a spectrogram and visualize it.

In [ ]:
def compute_mel_spectrogram(wav, sr=16000, n_mels=128, fixed_shape=(128, 128)):
    """
    Compute mel spectrogram with fixed dimensions
    """
    # Standardize length
    wav = standardize_audio_length(wav, sr=sr)
    
    # Compute mel spectrogram
    mel_spec = librosa.feature.melspectrogram(
        y=wav, sr=sr, n_fft=2048, hop_length=512, n_mels=n_mels)
    
    # Convert to decibels
    mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Resize to fixed dimensions if needed
    if fixed_shape:
        mel_spec = librosa.util.fix_length(mel_spec, size=fixed_shape[1], axis=1)
        
    return mel_spec

In [ ]:
def augment_audio(wav, sr=16000, apply_prob=0.5):
    """
    Apply random augmentations to audio data
    """
    # Random pitch shift between -2 and 2 semitones
    if np.random.random() < apply_prob:
        n_steps = np.random.uniform(-2, 2)
        wav = librosa.effects.pitch_shift(wav, sr=sr, n_steps=n_steps)
    
    # Random time stretching between 0.8x and 1.2x
    if np.random.random() < apply_prob:
        rate = np.random.uniform(0.8, 1.2)
        wav = librosa.effects.time_stretch(wav, rate=rate)
    
    # Add random noise
    if np.random.random() < apply_prob/2:  # Lower probability for noise
        noise_level = np.random.uniform(0, 0.005)
        noise = np.random.normal(0, noise_level, len(wav))
        wav = wav + noise
        
    # Ensure the amplitude is within [-1, 1]
    wav = np.clip(wav, -1, 1)
    
    return wav

## 6. Class Distribution

Analyze the distribution of chords in the Guitar Chords V2 dataset.

In [ ]:
# 6. Class Distribution
import glob

def get_all_files(dataset_path, chords):
    files = []
    labels = []
    for chord in chords:
        chord_path = os.path.join(dataset_path, chord, '*.wav')
        chord_files = glob.glob(chord_path)
        files += chord_files
        labels += [chord] * len(chord_files)
    return files, labels

# Get train and test files
train_files, train_labels = get_all_files(os.path.join(GUITARCHORDS_PATH, 'Training'), chords)
test_files, test_labels = get_all_files(os.path.join(GUITARCHORDS_PATH, 'Test'), chords)

# Combine and create a DataFrame
all_labels = train_labels + test_labels
df = pd.DataFrame({'Chord': all_labels})

# Plot
plt.figure(figsize=(10,6))
sns.countplot(data=df, x='Chord', order=chords)
plt.title('Chord Distribution in Guitar Chords V2 Dataset')
plt.xlabel('Chord')
plt.ylabel('Count')
plt.show()